 # __Classification of  COVID-19 on CT scans using Deep Learning__

## __ResNet50, ResNet50r, DenseNet121, MobileNet_v3_large and CaiT_xxs_24_224__


Image classification of CT scans in one of three classes: CAP, Covid, NonCovid


Datasets sourced from: 
* https://www.kaggle.com/maedemaftouni/large-covid19-ct-slice-dataset - Curated COVID-19 CT scan dataset from 7 public datasets
* https://data.mendeley.com/datasets/3y55vgckg6/2 - COVID-19 and common pneumonia chest CT dataset (only common pneumonia cases used)

<h2>Table of Contents</h2>
    <ul>
    <li><a href="#Section_1">Import libraries and supporting functions </a></li>
    <li><a href="#Section_2"> Exploring the dataset </a></li>
    <li><a href="#Section_3"> Build the experimental setup </a></li>
    <li><a href="#Section_4"> Training </a></li>
    <li><a href="#Section_5"> Evaluation on test dataset </a></li>
    <li><a href="#Section_6"> Image inference on test dataset </a></li>
    </ul>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

__Verify the GPU's available and their capacity__

In [ ]:
!nvidia-smi

Install required libraries

## __Importing Libraries supporting functions and methods__

In [3]:
# Library required to run transformer Class-Attention in Image Transformers (CaiT)
%%capture
!pip install timm 

In [4]:
# Use to compute AUROC
%%capture
!pip install torchmetrics

In [5]:
import os
import time
from datetime import date
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline
torch.manual_seed(123)

print('Using PyTorch version', torch.__version__)

Using PyTorch version 1.12.0+cu113


### __Change directory__

In [6]:
os.chdir('/content/drive/MyDrive/INM363_classification/classification')

In [11]:
from covid_class_dataset import CovidDataset, split_dataset, get_transform, get_vt_transform
from train_class import get_classification_model, main
from plotting import training_stats, show_predictions, show_misses
from evaluation_metrics import predictions, Metrics, misclassification, train_metrics

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## __Exploring the dataset__

### The dataset has three classes:

* COVID - Corona Virus Disease
* CAP - Community-Acquired Pneumonia 
* Non-covid - Healthy Lungs

In [ ]:
# Decompress the dataset file
!unzip curated_data.zip

In [ ]:
# Directory to access the images
data_dir = os.path.join(os.getcwd(), 'curated_data')
print(data_dir)

In [ ]:
class_names = ['cap', 'covid','non_covid']

In [ ]:
data =[]
data_id = {}
paths = {}
for idx, name, in enumerate (class_names):
    path = os.path.join(data_dir, name)
    l_dir = sorted(os.listdir(path))
    data_id[name] = l_dir
    paths[name] = path
    for file in l_dir:
        file_path = os.path.join(path, file)
        data.append([name, file_path, file])

In [ ]:
# Collates the paths and classes of each image in a dataframe for further processing
data_table = pd.DataFrame(data, columns=['image_label', 'image_path', 'image_id'])

In [ ]:
for k,v in data_id.items():
    print(f'Directory {k}, contains {len(data_id[k])}, instances')

In [ ]:
data_table.image_label.value_counts()

In [ ]:
data_table.info()

### __Classes distribution__

Here we will build a histogram to visualise the number of images per class.

In [ ]:
sns.set(style='whitegrid')
fig = plt.figure(figsize=(4,3))
dist = data_table.image_label.value_counts().plot.bar(color= "Green", alpha= 1)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Counts", fontsize=14)
plt.xlabel("Class")
plotname = 'class_dist.eps'
plt.savefig(os.path.join(os.getcwd(), plotname), bbox_inches='tight',
            format='eps', dpi=1200)

We can address the class imbalance by 
* splitting the dataset by stratifiying the images by the image_label
* using weighted cross entropy
* evaluating the performance of models with metrics that penalise misclassification of the minority class



### __Visualising raw images__

Here we will visualise the CT scan slices before applying augmentations

In [ ]:
sns.set(style='white', font_scale=1.2)
np.random.seed(123)
c = 1
eps = False
caps_id =[]
covid_id =[]
non_covid_id = []


fig = plt.figure(figsize = (20,20))
for i in range(9):
    if i < 3:
        idx = np.random.choice(data_table[data_table['image_label'] == 'cap'].index.values)
        file = data_table.iloc[idx]['image_path'] 
        # color = 'blue'
        caps_id.append(data_table.iloc[idx]['image_id'].strip('.png' or '.jpg'))

    elif 6 > i >2:
        idx = np.random.choice(data_table[data_table['image_label'] == 'covid'].index.values)
        file = data_table.iloc[idx]['image_path']
        # color = 'red'
        covid_id.append(data_table.iloc[idx]['image_id'].strip('.png'))

    else:
        idx = np.random.choice(data_table[data_table['image_label'] == 'non_covid'].index.values)
        file = data_table.iloc[idx]['image_path']
        # color = 'green'
        non_covid_id.append(data_table.iloc[idx]['image_id'].strip('.png'))

    img = Image.open(file)
    img_= img.resize((256,256))

    ax = fig.add_subplot(1, 9, c)
    c+=1
    ax.imshow(img_, cmap = 'gray')

    ax.set_title(data_table.iloc[idx].image_label, fontsize = 14, color = 'navy')
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])

print('CAPs ids', caps_id)
print('COVID ids', covid_id)
print('COVID ids', non_covid_id)


# Save image in eps format
if eps: 
    plotname = 'class3.eps'
    plt.savefig(os.path.join(os.getcwd(), plotname), bbox_inches='tight',
                format='eps', dpi=300)
plt.tight_layout()
plt.show()


## __Build the experimental setup__

To buil the setup we will use five Deep Learning (DL) neural networks::

* ResNet-50
* ResNet-50r
* DenseNet-121
* MobileNet-v3-large
* CaiT

We will train the neural network to optimize the loss functions (objective):

* Cross entropy
* Weighted cross entropy (added penalisation for the class imbalance)

Finally, we will optimize the objective function by using the optimizer
* Adam
* AdamW

In total, we will have 20 experiments as a result of each of the combinations.

In [ ]:
exps =[i for i in range(1, 21)]
exp_models = sorted(['resnet50', 'resnet50r', 'densenet121', 'mobilenet_v3_l', 'cait_24_224'] * 4)
exp_optim = ['Adam', 'AdamW'] * 10

In [ ]:
exp_loss=[]
for i in range (1,21,4):
    if i and i+1 in exps:
        exp_loss.extend(['CE']*2)
    if i+2 and i+3 in exps:
        exp_loss.extend(['wCE']*2)

In [ ]:
setup = pd.DataFrame({'Exp':exps,'Net': exp_models, 'Loss': exp_loss, 'Optim':exp_optim })

### __Experimental setup__


Experiment	 |Architecture  | Loss	 | Optimzer
-----|--------------|--------|--------
1	 |CaiT          |   CE   | Adam
2	 |CaiT          |	CE	 | AdamW
3	 |CaiT          |   wCE	 | Adam
4	 |CaiT          |   wCE	 | AdamW
5	 |DenseNet-121  |	CE	 | Adam
6	 |DenseNet-121  |   CE	 | AdamW
7	 |DenseNet-121  |   wCE	 | Adam
8	 |DenseNet-121	|   wCE	 | AdamW
9	 |MobileNet-v3-l|	CE	 | Adam
10	 |MobileNet-v3-l|   CE	 | AdamW
11	 |MobileNet-v3-l|   wCE	 | Adam
12	 |MobileNet-v3-l|   wCE	 | AdamW
13	 |ResNet-50	    |   CE	 | Adam
14	 |ResNet-50	    |   CE	 | AdamW
15	 |ResNet-50	    |   wCE	 | Adam
16	 |ResNet-50	    |   wCE	 | AdamW
17	 |ResNet-50r	|   CE	 | Adam
18	 |ResNet-50r	|   CE	 | AdamW
19	 |ResNet-50r	|   wCE	 | Adam
20	 |ResNet-50r	|   wCE	 | AdamW

## __Training__

Here we will bild our dataset using the class constructor to convert the images and labels into tensors. The tensors will be passed to the dataloader which will batchify the data for the training. 

In [ ]:
if __name__ == '__main__':

    today = date.today()
    date2 = today.strftime('%B %d, %Y')

    # Split dataset

    train, val, test = split_dataset(data_table, label_col ='image_label', seed=123)

    print('Train images:', len(train))
    print('Validation images:',len(val))
    print('Test images: ', len(test))

    # Define experiment to train and the number of run

    exp = 17
    run = '10'

    # Extract the architecture, optimizer and loss from the setup table using index for exp 

    idx = setup.Exp[setup.Exp == exp].index[0]
    model = setup.iloc[idx]['Net']
    optim = setup.Optim.iloc[idx]
    loss = setup.Loss.iloc[idx]

    # Creating weights to input onto the wCE
    class_count ={}
    for i, lb in enumerate (sorted(np.unique(train.image_label))):
        class_count[i] = train.image_label.value_counts()[lb]

    torch.set_printoptions(precision=5)
    weights = 1/torch.tensor(list(class_count.values()))
    weights.to(device) 

    # Create the output directory

    output_dir = os.path.join(model, 'exp_'+ str(exp), run)
    try:
        os.makedirs(output_dir, exist_ok=False)
        print('Directory successfully created')
    except OSError as error:
        print('Directory already exist')
        
    #Create data class to conver the images into tensors to feed on to the dataloaders
        
    if model!= 'cait_24_224':
        train_dataset = CovidDataset(train, 'image_path', 'image_label',
                                     get_transform(train=True))
        val_dataset = CovidDataset(val, 'image_path', 'image_label', 
                                   get_transform(train=False))
        test_dataset = CovidDataset(test, 'image_path', 'image_label', 
                                    get_transform(train=False))

    else:
        train_dataset = CovidDataset(train, 'image_path', 'image_label',
                                    get_vt_transform(train=True))
        val_dataset = CovidDataset(val, 'image_path', 'image_label',
                                  get_vt_transform(train =False))
        test_dataset = CovidDataset(test, 'image_path', 'image_label',
                                    get_vt_transform(train=False))

    # training hyperparameters dictionary
    
    params = {'exp': exp, 'model': model, 'num_classes': 3, 'device': device,
          'train_dataset': train_dataset, 'val_dataset': val_dataset, 'batch_size': 8,
          'workers': 2, 'loss': loss, 'weights': weights, 'optimizer': optim, 'lr': 2e-5,
          'momentum': 0.9, 'weight_decay': 1e-4, 'epochs': 8, 'output_dir': output_dir}
    
    # Provides information about the model to train and time and date

    print("Today's date:", date2)
    t = time.localtime()
    current_time = time.strftime('%H:%M:%S', t)
    print('Current_time', current_time)
    print('Model:', model, '|', 'run:', run, '|', 'loss:', loss, '|', 'optimizer:', optim)
    
    stats_log = main(**params)
   


### __Visualizing the training and validation accuracy and loss__

We can visualize the training and validation accuray loss directly from the stats log or stats from the 
checkpoint

* training_stats(params['model'], hideplot=False, **stats_log)


* fig_res = training_stats(checkpoint['stats'], params['model'])

In [ ]:
w_name = 'model_final_' + str(params['epochs']) + '.pth'
checkpoint = torch.load(os.path.join(output_dir, w_name))

We will visualize the stats from  the stats log

In [ ]:
training_stats(params['model'], hideplot=False, **stats_log)

## __Evaluating on the test dataset__

First we will extract the validation and training accuracies, the maximum validation and training accuracy ad the number of epochs at which the maximum was reached. 

In [ ]:
train_stats = checkpoint['stats']

In [ ]:
tr_acc, val_acc, max_acc, max_index, val_loss, min_loss, loss_idx = train_metrics(**train_stats)

To evaluate on te test set we require:

* Build the test loader 
* Build the model
* Upload the weigths obtained during training onto the model

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= params['batch_size'], shuffle=True)
model_ = get_classification_model(params['model'], params['num_classes'], pretrained=False)
model_.load_state_dict(checkpoint['model'], strict=False)

### __Obtain the prediction and probabilities__

In [ ]:
y_test, y_pred, y_prob, y_prob_tensor = predictions(model_.to(device), test_loader, device)

### Evaluate the model 

We will evaluate the performance of the experiments with the following metrics:

* Accuracy
* AUROC
* Balanced accuracy
* F1 
* F2 
* MCC
* Precision
* Sensitivity
* Specificity 

We also visualise the TP, TN, FP and FN in the confussion matrix and  a summary of the metrics in  the Classification report. 

In [ ]:
metrics = Metrics(y_test, y_pred, y_prob_tensor, class_names)

In [ ]:
metrics.cmatrix()

In [ ]:
print('Model:', params['model'],'|', 'run:', run, '|', 'loss:',params['loss'],'|', 'Optimizer:',
params['optimizer'])
print('Classification report', metrics.classification_rep(), sep='\n')

In [ ]:
macro_prec, recall, f1, f2, f1_class, f2_class = metrics.macro_average()

In [ ]:
micro_prec, micro_rec, micro_f1 = metrics.micro_average()
sens, spec, (tp, tn, fp, fn) = metrics.sensitivity_specificity()

In [ ]:
metrics_ = [['Date', date.today()], ['Exp', params['exp']], ['Run', run],
            ['Architecture', params['model']], ['Loss', params['loss']],
            ['Optimizer', params['optimizer']], ['Accuracy', micro_f1],
            ['BA', metrics.balanced_acc()],
            ['MCC' , metrics.mcc()],
            ['F1 macro', f1],
            ['Sensitivity', recall],
            ['Precision', macro_prec],
            ['Specificity', round(np.mean(spec),4)],
            ['F2', f2], ['AUROC', metrics.auroc()],
            ['F1 per class', f1_class],
            ['F2 per class', f2_class],
            ['Sensitivity per class', sens], ['Specificity per class', spec],
            ['Misses', fp + fn], ['FN', fn], ['FP', fp],
            ['tr_acc', tr_acc], ['val_acc', val_acc], 
            ['Max acc', max_acc], ['Max epoch', max_index+1],
            ['val_loss', val_loss], ['best loss', float(min_loss)],
            ['loss_idx', loss_idx +1]]


Create a dataframe and save the results in csv format

In [ ]:
df_metrics = pd.DataFrame(metrics_, columns= ['Metric','Value'],
                          index = [i for i in range(len(metrics_))])
print("Today's date:", today.strftime('%B %d, %Y'))
print('Model:', model,'|', 'run:', run, '|', 'loss:',loss,'|', 'Optimizer:',
          params['optimizer'])
print('Additional metrics')
df_metrics

In [ ]:
save_f = 'exp_'+  str(exp)+'_run_'+ run +'.csv'
outdir = params['output_dir']
df_metrics.to_csv(os.path.join(outdir, save_f))

##  __Inference - Showing prediction on test dataset__

In [ ]:
sns.set(style='white', font_scale=1)

### __Show predictions__

In [ ]:
fig = plt.figure(figsize=(20, 20))
show_predictions(model_, test_loader, class_names, device= torch.device('cpu'), cait = False,
                 outdir=None)

### __Show misclassifications__

In [ ]:
misses_data = misclassification(y_test, y_pred, y_prob, test) 

# Save the paths to the misclassified images in a csv file
misses_data.to_csv(output_dir + '/misses.csv')

In [ ]:
paths = show_misses(misses_data, class_names, 8) 

In [ ]:
# Create alist of the incorrect ids for visualisation
incorrect_ids = []
for path in misses_data['image_path']:
    im_id = path.replace('/content/drive/MyDrive/INM373_classification/curated_data/', '')
    incorrect_ids.append(im_id)
incorrect_ids

In [ ]:
# Transpose  the evaluation results and saved them into csv file
dft= df_metrics.T
df = dft.rename(columns=dft.iloc[0])
fname = '/dftranspose.csv'
outdir = params['output_dir']
df.to_csv(outdir + fname)

In [ ]:
# Clean the cache
torch.cuda.empty_cache()